In [ ]:
import os, json
import yaml


# root = '/private/home/delong/workspace/subobjects-VLM/runs/pixmo_cap/Llama-3_2-1B'
# steps = 8283


root = '/private/home/delong/workspace/subobjects-VLM/runs/clevr_caption/SmolLM2-135M-Instruct/patch'
steps = 2040

runs = os.listdir(root)

results = []
for run in runs:
    if run=='1206-1617-patch_4_per_side_random(256t-384px)-vae(384px)':
        continue
    print(run)
    trainer_states = os.path.join(root, run, f'runs/checkpoint-{steps}/trainer_state.json')
    if not os.path.exists(trainer_states):
        continue
    trainer_states = json.load(open(trainer_states))["log_history"]

    args = os.path.join(root, run, 'runs/args.yaml')
    args = yaml.load(open(args), Loader=yaml.FullLoader)
    tokenizer_family = args['visual_tokenizer_config'].split('/')[-2]
    results.append((tokenizer_family, args, trainer_states))
print(len(results))

In [ ]:


def sort_key(result):
    visual_tokenizer_config = result[1]['visual_tokenizer_config']
    if '/patch' in visual_tokenizer_config:
        return f"{int(visual_tokenizer_config.split('patch_')[-1].split('_')[0]):2}"
    else:
        return visual_tokenizer_config

# Sort the results based on the visual_tokenizer_config
results = sorted(results, key=sort_key)


tokenizer_families = list(set([r[0] for r in results]))
print(tokenizer_families)

for result in results:
    tokenizer_family, args, trainer_states = result
    print(tokenizer_family, args['visual_tokenizer_config'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 20))
cmaps = ['Purples', 'Blues', 'Greens', 'Reds']
for tokenizer_family in tokenizer_families:
    
    family_results = [r[1:] for r in results if r[0]==tokenizer_family]

    colors = plt.cm.get_cmap(cmaps.pop(0), len(family_results)+5)

    for i, (args, trainer_states) in enumerate(family_results):
        # print(args.keys())

        steps = [x['step'] for x in trainer_states]
        loss = [x['loss'] for x in trainer_states]
        loss_smoothed = np.convolve(loss, np.ones(32)/32, mode='valid')
        pad_size = (len(loss) - len(loss_smoothed)) // 2

        tokenizer_family, tokenizer = args['visual_tokenizer_config'].split('/')[-2:]
        tokenizer = tokenizer.replace('.json', '')

        max_visual_tokens = args['max_visual_tokens']
        tokenizer = f"{tokenizer} ({max_visual_tokens})"

        # plt.plot(steps, loss, color=colors(i+5), alpha=0.5, linewidth=1)
        plt.plot(
            steps[pad_size:-pad_size-1], loss_smoothed, 
            color=colors(i+5), 
            alpha=0.5, linewidth=1, label=tokenizer
            )

    plt.legend()
# plt.ylim(0.6, 1.8)
# plt.ylim(1.9,2.3)
plt.xlim(0, len(steps))
plt.show()